# HW18

In [5]:
%load_ext autoreload
%autoreload 2

## `RandomForestClassifierCustom`

In [53]:
import numpy as np

from custom_random_forest import RandomForestClassifierCustomMT, RandomForestClassifierCustomST
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

### Данные

In [8]:
X, y = make_classification(n_samples=100000)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### `RandomForestClassifierCustomST`

Основное решение задачи по параллелизации случайного леса.

Разбиение на процессы/потоки реализовано через `ProcessPoolExecutor`/`ThreadPoolExecutor` модуля `concurrent.futures`. Метод для обучения деревьев решений работает с одним деревом, а `ProcessPoolExecutor` делит все деревья между процессами внутри метода `fit`.

#### Время `fit`

In [12]:
%%timeit -r 5
random_forest_st = RandomForestClassifierCustomST(
    max_depth=30, n_estimators=30,
    max_features=2, random_state=42)
random_forest_st.fit(X_train, y_train, 1)

15.6 s ± 80.5 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [14]:
%%timeit -r 5
random_forest_st = RandomForestClassifierCustomST(
    max_depth=30, n_estimators=30,
    max_features=2, random_state=42)
random_forest_st.fit(X_train, y_train, 2)

8.83 s ± 37.1 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [ ]:
%%timeit -r 5
random_forest_st = RandomForestClassifierCustomST(
    max_depth=30, n_estimators=30,
    max_features=2, random_state=42)
random_forest_st.fit(X_train, y_train, 5)

5 s ± 57.9 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [ ]:
%%timeit -r 5
random_forest_st = RandomForestClassifierCustomST(
    max_depth=30, n_estimators=30,
    max_features=2, random_state=42)
random_forest_st.fit(X_train, y_train, 10)

4.41 s ± 74.6 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


#### Время `predict`

In [16]:
random_forest_st = RandomForestClassifierCustomST(
    max_depth=30, n_estimators=30,
    max_features=2, random_state=42)
random_forest_st.fit(X_train, y_train, 2)

RandomForestClassifierCustomST(max_depth=30, max_features=2, n_estimators=30,
                               random_state=42)

In [17]:
%%timeit -r 5
y_pred_st = random_forest_st.predict(X_test, 1)

108 ms ± 781 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [18]:
%%timeit -r 5
y_pred_1t = random_forest_st.predict(X_test, 2)

60 ms ± 2.06 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [49]:
%%timeit -r 5
y_pred_1t = random_forest_st.predict(X_test, 5)

29.8 ms ± 664 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [50]:
%%timeit -r 5
y_pred_1t = random_forest_st.predict(X_test, 10)

22.9 ms ± 616 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


### `RandomForestClassifierCustomMT`

Дополнительное решение задачи по параллелизации случайного леса - из интереса, чтобы поиграться с распределением задач по процессам и извлечением данных из них.

Разбиение на процессы/потоки реализовано через `multiprocessing.Process`/`threading.Thread` соответственно. При обучении леса создаётся список с порядковыми номерами деревьев, которые делится (по числу процессов) на равные подписки. Каждый подсписок передаётся в отдельный процесс, запускающий метод для обучения деревьев решений; внутри процесса просходит итерирация по подписку и обучение деревьев в цикле.

#### Разбиение по задач потокам.

In [ ]:
n_jobs = 8
n_estimators = 30

In [ ]:
def split_estimators(n_estimators, n_jobs):
    floordiv, mod = n_estimators // n_jobs, n_estimators % n_jobs
    idx = np.arange(floordiv, floordiv * n_jobs, floordiv)
    if mod:
        idx += np.append(
                np.zeros(n_jobs - mod, dtype='int8'),
                np.arange(1, mod))
    return np.split(np.arange(n_estimators), idx)

In [ ]:
split_estimators(n_estimators, n_jobs)

[array([0, 1, 2]),
 array([3, 4, 5]),
 array([6, 7, 8, 9]),
 array([10, 11, 12, 13]),
 array([14, 15, 16, 17]),
 array([18, 19, 20, 21]),
 array([22, 23, 24, 25]),
 array([26, 27, 28, 29])]

#### Время `fit`

Для 1 и 2 процессов явно видна разница в ~1.5 раза. При увеличении числа процессов кратного снижение времени обучения не происходит.

В основном решении, при использовании `ProcessPoolExecutor` такого не происходит, время кратно снижается при увеличении числа ппроцессов.

In [ ]:
%%timeit -r 5
random_forest_mt = RandomForestClassifierCustomMT(
    max_depth=30, n_estimators=30,
    max_features=2, random_state=42)
random_forest_mt.fit(X_train, y_train, 1)

14.9 s ± 177 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [ ]:
%%timeit -r 5
random_forest_mt = RandomForestClassifierCustomMT(
    max_depth=30, n_estimators=30,
    max_features=2, random_state=42)
random_forest_mt.fit(X_train, y_train, 2)

9.19 s ± 37.4 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [ ]:
%%timeit -r 5
random_forest_mt = RandomForestClassifierCustomMT(
    max_depth=30, n_estimators=30,
    max_features=2, random_state=42)
random_forest_mt.fit(X_train, y_train, 5)

9.13 s ± 130 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [24]:
%%timeit -r 5
random_forest_mt = RandomForestClassifierCustomMT(
    max_depth=30, n_estimators=30,
    max_features=2, random_state=42)
random_forest_mt.fit(X_train, y_train, 10)

13.5 s ± 163 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


#### Время `predict`

In [19]:
random_forest_mt = RandomForestClassifierCustomMT(
    max_depth=30, n_estimators=30,
    max_features=2, random_state=42)
random_forest_mt.fit(X_train, y_train, 2)

RandomForestClassifierCustomMT(max_depth=30, max_features=2, n_estimators=30,
                               random_state=42)

In [20]:
%%timeit -r 5
y_pred_mt = random_forest_mt.predict(X_test, 1)

239 ms ± 10.1 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [21]:
%%timeit -r 5
y_pred_mt = random_forest_mt.predict(X_test, 2)

136 ms ± 2.79 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


## `OpenFasta`

In [35]:
from bio_files_processor import FastaRecord, OpenFasta

### `FastaRecord`

Создадим экземпляр `FastaRecord`:

In [36]:
fasta_exmp0 = FastaRecord('GTD323452', '5S_rRNA NODE_272_length_223_cov_0.720238:18-129(+)', 'ACGGCCATAGGACTTTGAAAGCACCGCATCCCGTCCGATCTGCGAAGTTAACCAAGATGCCGCCTGGTTAGTACCATGGTGGGGGACCACATGGGAATCCCTGGTGCTGTG')
print(fasta_exmp0)

ID: GTD323452
Description: 5S_rRNA NODE_272_length_223_cov_0.720238:18-129(+)
Sequence: ACGGCCATAGGACTTTGAAAGCACCGCATCCCGTCCGATCTGCGAAGTTAACCAAGATGCCGCCTGGTTAGTACCATGGTGGGGGACCACATGGGAATCCCTGGTGCTGTG



`FastaRecord`, у которого нет ID или описания, считается `False`:

In [37]:
empty_fasta_exmp0 = FastaRecord('', '', '')
bool(empty_fasta_exmp0)

False

In [38]:
empty_fasta_exmp1 = FastaRecord('', '', 'ACGGCCATAGGACTTTGAAAGCACCGCATCCCGTCCGATCTGCGAAGTTAACCAAGATGCCGCCTGGTTAGTACCATGGTGGGGGACCACATGGGAATCCCTGGTGCTGTG')
bool(empty_fasta_exmp0)

False

Если есть ID или описание - булева проверка вернёт `True`:

In [39]:
fasta_exmp1 = FastaRecord('GTD323452', '', '')
bool(fasta_exmp1)

True

In [40]:
fasta_exmp1 = FastaRecord('', '5S_rRNA NODE_272_length_223_cov_0.720238:18-129(+)', '')
bool(fasta_exmp1)

True

### Чтение fasta-файлов

#### Чтение файлов

`OpenFasta` позволяет читать файлы за счёт метода `file_open`:

In [42]:
fasta = OpenFasta('example_data/example_fasta.fasta')

fasta.file_open()
print(fasta.read_record())
fasta.file_close()

print(f'Instance handler is closed: {fasta.handler.closed}')

ID: GTD323452
Description: 5S_rRNA NODE_272_length_223_cov_0.720238:18-129(+)
Sequence: ACGGCCATAGGACTTTGAAAGCACCGCATCCCGTCCGATCTGCGAAGTTAACCAAGATGCCGCCTGGTTAGTACCATGGTGGGGGACCACATGGGAATCCCTGGTGCTGTG

Instance handler is closed: True


Убедимся, что файл закрыт на чтение:

In [43]:
print(fasta.read_record())

ValueError: I/O operation on closed file.

`OpenFasta` также является итерируемым объектом, что позволяет итерироваться по записям в цикле `for`:

In [44]:
for line in OpenFasta('example_data/example_fasta.fasta'):
    print(line)

ID: GTD323452
Description: 5S_rRNA NODE_272_length_223_cov_0.720238:18-129(+)
Sequence: ACGGCCATAGGACTTTGAAAGCACCGCATCCCGTCCGATCTGCGAAGTTAACCAAGATGCCGCCTGGTTAGTACCATGGTGGGGGACCACATGGGAATCCCTGGTGCTGTG

ID: GTD678345
Description: 16S_rRNA NODE_80_length_720_cov_1.094737:313-719(+)
Sequence: TTGGCTTCTTAGAGGGACTTTTGATGTTTAATCAAAGGAAGTTTGAGGCAATAACAGGTCTGTGATGCCCTTAGATGTTCTGGGCCGCACGCGCGCTACACTGAGCCCTTGGGAGTGGTCCATTTGAGCCGGCAACGGCACGTTTGGACTGCAAACTTGGGCAAACTTGGTCATTTAGAGGAAGTAAAAGTCGTAACAAGGT

ID: GTD174893
Description: 16S_rRNA NODE_1_length_2558431_cov_75.185164:2153860-2155398(+)
Sequence: TTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAACGGTAACAGGAAACAGCTTGCTGTTTCGCTGACGAGTGGGAAGTAGGTAGCTTAACCTTCGGGAGGGCGCTTACCACTTTGTGATTCATGACTGGGGTGAAGTCGTAACAAGGTAACCGTAGGGGAACCTGCGGTTGGATCACCTCCTT

ID: GTD906783
Description: 16S_rRNA NODE_1_length_2558431_cov_75.185164:793941-795479(-)
Sequence: TTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAACGGTAACAGGAAACAGCTTGCTGTTTCGC

#### Чтение с помощью конструкции `with-as`

`OpenFasta` реализует протокол контестного менеджера:

In [45]:
with OpenFasta('example_data/example_fasta.fasta') as fasta:
    print('__enter__' in dir(fasta))
    print('__exit__' in dir(fasta))

True
True


`OpenFasta` позволяет читать каждую последовательность отдельно с помощью метода `read_record()`:

In [46]:
with OpenFasta('example_data/example_fasta.fasta') as fasta:
    print(fasta.read_record())

ID: GTD323452
Description: 5S_rRNA NODE_272_length_223_cov_0.720238:18-129(+)
Sequence: ACGGCCATAGGACTTTGAAAGCACCGCATCCCGTCCGATCTGCGAAGTTAACCAAGATGCCGCCTGGTTAGTACCATGGTGGGGGACCACATGGGAATCCCTGGTGCTGTG



а также итерироваться по записям в цикле:

In [47]:
with OpenFasta('example_data/example_fasta.fasta') as fasta:
    for line in fasta:
        print(line)

ID: GTD323452
Description: 5S_rRNA NODE_272_length_223_cov_0.720238:18-129(+)
Sequence: ACGGCCATAGGACTTTGAAAGCACCGCATCCCGTCCGATCTGCGAAGTTAACCAAGATGCCGCCTGGTTAGTACCATGGTGGGGGACCACATGGGAATCCCTGGTGCTGTG

ID: GTD678345
Description: 16S_rRNA NODE_80_length_720_cov_1.094737:313-719(+)
Sequence: TTGGCTTCTTAGAGGGACTTTTGATGTTTAATCAAAGGAAGTTTGAGGCAATAACAGGTCTGTGATGCCCTTAGATGTTCTGGGCCGCACGCGCGCTACACTGAGCCCTTGGGAGTGGTCCATTTGAGCCGGCAACGGCACGTTTGGACTGCAAACTTGGGCAAACTTGGTCATTTAGAGGAAGTAAAAGTCGTAACAAGGT

ID: GTD174893
Description: 16S_rRNA NODE_1_length_2558431_cov_75.185164:2153860-2155398(+)
Sequence: TTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAACGGTAACAGGAAACAGCTTGCTGTTTCGCTGACGAGTGGGAAGTAGGTAGCTTAACCTTCGGGAGGGCGCTTACCACTTTGTGATTCATGACTGGGGTGAAGTCGTAACAAGGTAACCGTAGGGGAACCTGCGGTTGGATCACCTCCTT

ID: GTD906783
Description: 16S_rRNA NODE_1_length_2558431_cov_75.185164:793941-795479(-)
Sequence: TTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAACGGTAACAGGAAACAGCTTGCTGTTTCGC

или прочесть все записи сразу с помощью метода `read_records()`:

In [48]:
with OpenFasta('example_data/example_fasta.fasta') as fasta:
    print(fasta.read_records())

[ID: GTD323452
Description: 5S_rRNA NODE_272_length_223_cov_0.720238:18-129(+)
Sequence: ACGGCCATAGGACTTTGAAAGCACCGCATCCCGTCCGATCTGCGAAGTTAACCAAGATGCCGCCTGGTTAGTACCATGGTGGGGGACCACATGGGAATCCCTGGTGCTGTG
, ID: GTD678345
Description: 16S_rRNA NODE_80_length_720_cov_1.094737:313-719(+)
Sequence: TTGGCTTCTTAGAGGGACTTTTGATGTTTAATCAAAGGAAGTTTGAGGCAATAACAGGTCTGTGATGCCCTTAGATGTTCTGGGCCGCACGCGCGCTACACTGAGCCCTTGGGAGTGGTCCATTTGAGCCGGCAACGGCACGTTTGGACTGCAAACTTGGGCAAACTTGGTCATTTAGAGGAAGTAAAAGTCGTAACAAGGT
, ID: GTD174893
Description: 16S_rRNA NODE_1_length_2558431_cov_75.185164:2153860-2155398(+)
Sequence: TTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAACGGTAACAGGAAACAGCTTGCTGTTTCGCTGACGAGTGGGAAGTAGGTAGCTTAACCTTCGGGAGGGCGCTTACCACTTTGTGATTCATGACTGGGGTGAAGTCGTAACAAGGTAACCGTAGGGGAACCTGCGGTTGGATCACCTCCTT
, ID: GTD906783
Description: 16S_rRNA NODE_1_length_2558431_cov_75.185164:793941-795479(-)
Sequence: TTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAACGGTAACAGGAAACAGCTTGCTGTT

## `run_genscan`

Я не успел его сделать, поэтому вместо него здесь будет результат работы тестирующего скрипта.

In [52]:
!python -m pytest test_script.py

============================= test session starts =============================
platform win32 -- Python 3.11.8, pytest-8.1.1, pluggy-1.4.0
rootdir: d:\Users\d2707\Documents\Colab Notebooks\Курсы\Институт биоинформатики\Семестр 2\Python\Занятие 23 - Тестирование\HW18\HW5_Modules_Bredov
collected 12 items

test_script.py ............                                              [100%]

============================= 12 passed in 0.33s ==============================


## `AminoAcidSequence`/`DNASequence`/`RNASequence`

In [26]:
from ultimate_bioinf_tool import (BiologicalSequence,
                                  BiologicalSequenceInitializer,
                                  NucleicAcidSequence,
                                  RNASequence,
                                  DNASequence,
                                  AminoAcidSequence)

### 1 метод который работает с РНК

In [27]:
RNASequence('AUGC').check_alphabet()

True

### 1 метод который работает с белками

In [28]:
AminoAcidSequence('ARK').content_check()

{'A': 33.333333333333336,
 'R': 33.333333333333336,
 'N': 0.0,
 'D': 0.0,
 'C': 0.0,
 'Q': 0.0,
 'E': 0.0,
 'G': 0.0,
 'H': 0.0,
 'I': 0.0,
 'L': 0.0,
 'K': 33.333333333333336,
 'M': 0.0,
 'F': 0.0,
 'P': 0.0,
 'S': 0.0,
 'T': 0.0,
 'W': 0.0,
 'Y': 0.0,
 'V': 0.0}

### Какой тип данных возвращает метод `complement` который вызвали на последовательности ДНК

In [29]:
type(DNASequence('ATGC').complement())

ultimate_bioinf_tool.DNASequence

### Остальные примеры

In [30]:
DNASequence(['A','T','G','C']).gc_content()

0.5

In [31]:
DNASequence('AtGCTt').transcribe()

RNASequence(AuGCUu)

In [32]:
RNASequence('ATGC')

TypeError: Sequence does not match alphabet for RNASequence(None). Please, use only: {'c', 'A', 'U', 'g', 'u', 'a', 'G', 'C'}.

In [33]:
AminoAcidSequence('ARK')

AminoAcidSequence(ARK)

In [34]:
AminoAcidSequence('AUR')

TypeError: Sequence does not match alphabet for AminoAcidSequence(None). Please, use only: {'R', 'G', 'A', 'E', 'N', 'Q', 'F', 'L', 'P', 'D', 'C', 'S', 'W', 'T', 'Y', 'M', 'H', 'K', 'I', 'V'}.